In [1]:
# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")


LoadError: InitError: PyError (PyImport_ImportModule

The Python package matplotlib could not be imported by pyimport. Usually this means
that you did not install matplotlib in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/bin/python3.8

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the matplotlib module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.buil

In [2]:
function Laplacian_2D!(P₁,P₂,A₁,A₂,Δx)
    B₁ = CircularArrays.CircularArray(A₁);
    B₂ = CircularArrays.CircularArray(A₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j) = Tuple(😄)
        P₁[i,j] = @fastmath (-B₁[i+2,j] + 16B₁[i+1,j] 
            + 16B₁[i-1,j] -B₁[i-2,j] 
            -B₁[i,j+2] + 16B₁[i,j+1] 
            + 16B₁[i,j-1] -B₁[i,j-2] 
            - 60B₁[i,j]) /(12Δx^2);
        P₂[i,j] = @fastmath (-B₂[i+2,j] + 16B₂[i+1,j] 
            + 16B₂[i-1,j] -B₂[i-2,j] 
            -B₂[i,j+2] + 16B₂[i,j+1] 
            + 16B₂[i,j-1] -B₂[i,j-2] 
            - 60B₂[i,j]) /(12Δx^2);  
    end
    return nothing
end



function Cores_2D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            norm₁ = (angle[😄,🥪] + π)/(2π)
            norm₂ = (angle[😄+1,🥪] + π)/(2π)
            norm₃ = (angle[😄+1,🥪+1] + π)/(2π)
            norm₄ = (angle[😄,🥪+1] + π)/(2π)

            θ₁ = min(abs(norm₂ - norm₁), 1 - abs(norm₂ - norm₁))
            θ₂ = min(abs(norm₃ - norm₂), 1 - abs(norm₃ - norm₂))
            θ₃ = min(abs(norm₄ - norm₃), 1 - abs(norm₄ - norm₃))
            θₛ = θ₁ + θ₂ + θ₃

            if θₛ > accept 
                append!(s,[[😄,🥪]])
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]

            if diffᵣ == 0 && diffₛ == 1
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0
                count += 1
            end
        end
    end

    num = length(s) - count

    return num
end



function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j) = Tuple(😄)
        @fastmath moo[i,j] = sqrt(A₁[i,j]^2 + A₂[i,j]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j) = Tuple(😄)
        angle[i,j] = @fastmath atan(A₂[i,j],A₁[i,j]);
    end
    return nothing
end

function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [3]:
function PQfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j]);
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j]);
    end
    return nothing
end

function PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    PQfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))
    if time < 250
        PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)
    else 
        PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt * 250/time,1,fₐ)
    end

    return nothing
end



PQupdate_2D! (generic function with 1 method)

In [4]:
function EQCDfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j])
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j])
    end
    return nothing
end


function EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s)

    n = 1

    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);
    if time < 1.8
        EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)
    else
        EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt* (1.8/time)^3.34,λ,ηₓ,n)
    end

    return nothing
end

EQCDupdate_2D! (generic function with 1 method)

In [5]:
function LLaplacian_2D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j) = Tuple(😄)
        P[i,j] = @fastmath (-B[i+2,j] + 16B[i+1,j] 
            + 16B[i-1,j] -B[i-2,j] 
            -B[i,j+2] + 16B[i,j+1] 
            + 16B[i,j-1] -B[i,j-2] 
            - 60B[i,j]) /(12Δx^2);
    end
    return nothing
end


function Lfupdate_2D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j) = Tuple(😄)
        F[i,j] = @fastmath M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j];
    end
    return nothing
end


function LAupdate_2D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j) = Tuple(😄)
        A[i,j] = @fastmath A[i,j] .+ Δt .* (Ȧ[i,j] .+ 0.5Δt .* F[i,j])
    end
    return nothing
end


function Lvelupdate_2D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j) = Tuple(😄)
        Ȧ[i,j] = @fastmath Ȧ[i,j] .+ 0.5Δt .* (F[i,j] .+ M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j])
    end
    return nothing
end


function Lupdate_2D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s)

    #ηₓ,η = Lηtime(time,fₐ);
    n = 1

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_2D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_2D!(A,Δt,Ȧ,F)

    LLaplacian_2D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_2D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_2D! (generic function with 1 method)

In [6]:


function FPQrun_2D!(N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i)
    #700 images on per 5000 updates
    time = t₀

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 50

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []

    axenergy = []


    for lo ∈ 1:round((250- t₀)/Δt,digits = 0)
        time = round(time,digits = 4);


        if lo % x == 0
            #125 pictures
            mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            angler!(angle,A₁,A₂);

            strng = Cores_2D!(N,angle,thr)
            append!(tracker,time)
            append!(cores,strng)

            # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")
            
            f_image = FFTW.fft(moo)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

            append!(saxion,[Abin])
            
            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


            append!(axion,[Abins])
            

            #PyPlot.imsave("PQEpoch/"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")


            append!(axenergy,mean( ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)).^2 .* fₐ^2 ))
        end


        PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)
        time = time + Δt

    end
    print(time)
    tim = time
    lo = round((250-t₀)/Δt,digits = 0)*10 + 1

    x = 500

    while tim < 800
        #make a note a 280 where we get PQ transition
        tim = round(tim,digits = 5);


        if lo % x == 0
            mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            angler!(angle,A₁,A₂);


            strng = Cores_2D!(N,angle,thr)
            append!(tracker,tim)
            append!(cores,strng)

            #PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")
            
            f_image = FFTW.fft(moo)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

            append!(saxion,[Abin])
            
            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


            append!(axion,[Abins])
            
            

            #PyPlot.imsave("PQEpoch/"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")

            append!(axenergy,mean( ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)).^2 .* fₐ^2 ))
        end


        PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
        tim = tim + Δt * 250/tim
        lo += 1
    end
        

    return tracker, cores, saxion, axion, axenergy
end

FPQrun_2D! (generic function with 1 method)

In [7]:


function FErun_2D!(N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s)


    time = t₁

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N);
    angle = zeros(N,N);
    angler!(angle,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = 10

    tracker = []
    cores = []
    thr = 1
    lo = 1
    strng = 1

    saxion = []
    axion = []
    axenergy = []


    for lo ∈ 1:round((1.8-t₁)/Δt,digits = 0)
    #while strng != 0

        time = round(time,digits = 3);
        if lo % x == 0
            #18 pictures
            mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            angler!(angle,A₁,A₂);

            strng = Cores_2D!(N,angle,thr)
            append!(tracker,time)
            append!(cores,strng)

            # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

            f_image = FFTW.fft(moo)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")
            
            append!(saxion,[Abin])

            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")
            
            append!(axion,[Abins])

            #PyPlot.imsave("QCD/"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
            append!(axenergy,mean( ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)).^2 .* fₐ^2 ))

        end
        EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s)
        time = time + Δt
        #lo += 1
        
    end

    lo = round((1.8-t₁) /Δt,digits = 0) + 1


    while strng != 0 
        time = round(time,digits = 6);
        if lo % x == 0
            #18 pictures
            mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            angler!(angle,A₁,A₂);

            strng = Cores_2D!(N,angle,thr)
            append!(tracker,time)
            append!(cores,strng)

            # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

            f_image = FFTW.fft(moo)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")
            
            append!(saxion,[Abin])

            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")
            
            append!(axion,[Abins])

            #PyPlot.imsave("QCD/"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
            append!(axenergy,mean( ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)).^2 .* fₐ^2 ))

        end
        EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt* (1.8/time)^3.34,time,fₐ,r,s)
        time = time + Δt* (1.8/time)^3.34
        lo += 1
    end

    stat = 0

    while stat < 0.2
        time = round(time,digits = 6);
        if lo % x == 0
            #18 pictures
            mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            angler!(angle,A₁,A₂);

            strng = Cores_2D!(N,angle,thr)
            append!(tracker,time)
            append!(cores,strng)

            # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

            f_image = FFTW.fft(moo)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")
            
            append!(saxion,[Abin])

            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")
            
            append!(axion,[Abins])

            #PyPlot.imsave("QCD/"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
            append!(axenergy,mean( ((Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)).^2 .* fₐ^2 ))

        end
        EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt* (1.8/time)^3.34,time,fₐ,r,s)
        time = time + Δt* (1.8/time)^3.34
        stat += Δt* (1.8/time)^3.34
        lo += 1
    end
    
    print(time)
    tim = time

    lo = lo*10

    x = 100

    Ȧ = zeros(N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N);

    F = zeros(N,N);



    LLaplacian_2D!(M,angle,Δx)

    while tim < 7
        
        tim = round(tim,digits = 6);
        if lo % x == 0
            #13 pictures
            #mooing!(moo,A₁,A₂);
            #println(moo[1,1])
            #setting!(moo);
            #angler!(angle,A₁,A₂);

            strng = Cores_2D!(N,angle,thr)
            append!(tracker,tim)
            append!(cores,strng)

            # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

            # f_image = FFTW.fft(moo)
            # f_images = (abs.(f_image)).^2
            # f_images = collect(Iterators.flatten(f_images))
   
            # Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            # Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
            # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")
            
            #append!(saxion,[Abin])

            f_image = FFTW.fft(angle)
            f_images = (abs.(f_image)).^2
            f_images = collect(Iterators.flatten(f_images))
   
            Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
            Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)
         
            # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
            # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")
            
            append!(axion,[Abins])

            #PyPlot.imsave("QCD/"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
            append!(axenergy,mean( Ȧ.^2 .* fₐ^2 ))

        end
        Lupdate_2D!(angle,Ȧ,M,F,Δx,Δt,time,fₐ,s)
        tim = tim + Δt* (1.8/tim)^3.34
        lo += 1
    end
    
    time = time - 0.2

    return tracker,cores,saxion,axion,axenergy,time
end




FErun_2D! (generic function with 1 method)

In [11]:
using Pkg
ENV["PYTHON"]="/usr/bin/python3.8"
Pkg.build("PyCall")


using CircularArrays
using JLD
using FFTW
#Pkg.add("PyCall")
#Pkg.build("PyCall")
#Pkg.add("SciPy")
#Pkg.add("Conda")
using Conda
Conda.add("scipy")
using PyCall
stats = pyimport("scipy.stats")



    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/299304989a5e6473d985212c28928899c74e9421/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/169bb8ea6b1b143c5cf57df6d34d022a7b60c6db/build.log`
┌ Info: Running `conda install -y scipy` in root environment
└ @ Conda /home/shan/.julia/packages/Conda/sNGum/src/Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



PyObject <module 'scipy.stats' from '/home/shan/.local/lib/python3.8/site-packages/scipy/stats/__init__.py'>

In [ ]:


@time begin
for n ∈ 9:9

    
    
    N = 2^n;
    println(N)
    
    rₐ = range(15,stop = 24,length = 10);
    fₐ = 10 .^rₐ;
    r = 1
    s = 1
    
    
    for i ∈ 1:1
        
        t₀ = 0.0001;
        print(string(i)*" ")
        println(fₐ[i])
    
        Ȧ₁ = zeros(N,N);
        Ȧ₂ = zeros(N,N);
        Ȧ = zeros(N,N);
    
        μ,σ = 0, 0.1;
        C₁ = rand(Normal(μ,σ),N,N);
        C₂ = rand(Normal(μ,σ),N,N);
    
        Δx = round(8000/N,digits = 2)
        Δt = 0.004
    
        tracker,cores,saxion,axion,axenergy = FPQrun_2D!(N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ[i],i)

        
        save("PQend"*string(n)*".jld", "Real", C₁,"Imaginary",C₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)
        save("PQStrings"*string(n)*".jld","time",tracker,"number",cores)
        save("PQStats"*string(n)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)

    
    end
    
    
end
end

512
1 1.0e15


In [ ]:
@time begin

n = 9
N = 2^n


Δx = round(4/N,digits = 5)
Δt = 0.001 

rₐ = range(15,stop = 24,length = 10);
fₐ = 10 .^rₐ;

i = 1
for r ∈ 1:1
    for s ∈ 5:5
        C₁ = load("PQend"*string(n)*".jld")["Real"]
        C₂ = load("PQend"*string(n)*".jld")["Imaginary"]
        Ȧ₁ = load("PQend"*string(n)*".jld")["Realvel"]
        Ȧ₂ = load("PQend"*string(n)*".jld")["RealIm"]

        t₁ = 0.4


        tracker,cores,saxion,axion,axenergy,time = FErun_2D!(N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ[i],i,r,s)


        save("QCDend"*string(n)*string(r)*string(s)*".jld", "Real", C₁,"Imaginary",C₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)
        save("QCDStrings"*string(n)*string(r)*string(s)*".jld","time",tracker,"number",cores)
        save("QCDStats"*string(n)*string(r)*string(s)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy,"defects",time)   

    
    end
end
    
for r ∈ 2:5
    for s ∈ 1:1
        C₁ = load("PQend"*string(n)*".jld")["Real"]
        C₂ = load("PQend"*string(n)*".jld")["Imaginary"]
        Ȧ₁ = load("PQend"*string(n)*".jld")["Realvel"]
        Ȧ₂ = load("PQend"*string(n)*".jld")["RealIm"]

        t₁ = 0.4


        tracker,cores,saxion,axion,axenergy,time = FErun_2D!(N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ[i],i,r,s)


        save("QCDend"*string(n)*string(r)*string(s)*".jld", "Real", C₁,"Imaginary",C₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)
        save("QCDStrings"*string(n)*string(r)*string(s)*".jld","time",tracker,"number",cores)
        save("QCDStats"*string(n)*string(r)*string(s)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy,"defects",time)   

    
    end
end


end
